In [ ]:
import ee
import geemap.core as geemap

In [ ]:
ee.Authenticate()
ee.Initialize(project='flood-detection-463214')

In [ ]:
land_cover_1 = ee.Image('projects/flood-detection-463214/assets/17S_20180101-20190101')
land_cover_2 = ee.Image('projects/flood-detection-463214/assets/18S_20180101-20190101')
land_cover = ee.ImageCollection([land_cover_1, land_cover_2]).mosaic()

In [ ]:
flood_mask = ee.Image('projects/flood-detection-463214/assets/FloodExtentFlorence')
water_image = land_cover.eq(1).Or(land_cover.eq(4))
change_mask = flood_mask.unmask().And(water_image.Not()).selfMask()

In [ ]:
dataset = ee.ImageCollection('USDA/NAIP/DOQQ').filter(ee.Filter.date('2016-01-01', '2018-08-01'));
trueColor = dataset.select(['R', 'G', 'B']);
trueImage = trueColor.mosaic()

In [ ]:
se_nc = ee.Geometry.Rectangle([-80, 33.5, -76, 36])
map = geemap.Map()
map.addLayer(se_nc, {}, 'SE NC Rectangle')
map

In [ ]:
task = ee.batch.Export.image.toDrive(
    image=change_mask.unmask().clip(se_nc),
    description="flood_change_mask",
    region=se_nc,
    crs='EPSG:3857',
    scale=20,
    maxPixels=1e13
)
task.start()


In [ ]:
task = ee.batch.Export.image.toDrive(
    image=land_cover.unmask().clip(se_nc),
    description="optical_sc_nc",
    region=se_nc,
    crs='EPSG:3857',
    scale=20,
    maxPixels=1e13
)
task.start()


In [ ]:
task = ee.batch.Export.image.toDrive(
    image=trueImage.clip(se_nc),
    description="land_cover_mask",
    region=se_nc,
    crs='EPSG:3857',
    scale=20,
    maxPixels=1e13
)
task.start()
